#  P6 : Classifiez automatiquement des biens de consommation¶

# Classiffication

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from glob import glob

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D 
from tensorflow.keras.layers import GlobalAveragePooling1D 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical

# Warnings
import warnings
warnings.filterwarnings('ignore')


2023-05-12 17:58:01.276091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Étape 0 : Division du dataset en train et test

In [7]:
import pandas as pd

# Chargement du jeu de données
df = pd.read_csv('flipkart_com-ecommerce_sample_1050.csv')
# Ajout variable images
path_images = '/Users/danongohou/Desktop/P6/Images/'

# Ajout de nouvelles variables 
df['label_name'] = df['product_category_tree'].apply(lambda x: x.split('>>')[0][2:].strip().replace('&', 'and'))
df['image_path'] = [path_images + row for row in df['image']]


In [8]:

df = df[['image','image_path', 'label_name']]
df.shape

(1050, 3)

In [9]:
df['label_name'].value_counts() 

label_name
Home Furnishing                 150
Baby Care                       150
Watches                         150
Home Decor and Festive Needs    150
Kitchen and Dining              150
Beauty and Personal Care        150
Computers                       150
Name: count, dtype: int64

In [15]:
data = df.copy()
data.to_csv("data_cls.csv", index=False)
data.head()

,image,image_path,label_name
0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,/Users/danongohou/Desktop/P6/Images/55b85ea15a...,Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590.jpg,/Users/danongohou/Desktop/P6/Images/7b72c92c2f...,Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74.jpg,/Users/danongohou/Desktop/P6/Images/64d5d4a258...,Baby Care
3,d4684dcdc759dd9cdf41504698d737d8.jpg,/Users/danongohou/Desktop/P6/Images/d4684dcdc7...,Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7.jpg,/Users/danongohou/Desktop/P6/Images/6325b6870c...,Home Furnishing


In [19]:
data.image_path.value_counts() 

image_path
/Users/danongohou/Desktop/P6/Images/55b85ea15a1536d46b7190ad6fff8ce7.jpg    1
/Users/danongohou/Desktop/P6/Images/55195de3873fffaa9e37b041533d7305.jpg    1
/Users/danongohou/Desktop/P6/Images/c65fa0b38ab99792630468bd7dd26416.jpg    1
/Users/danongohou/Desktop/P6/Images/142994b421c052a6193052c6c8c3d076.jpg    1
/Users/danongohou/Desktop/P6/Images/c6adf41cd378b1f6a7ac62675b060411.jpg    1
                                                                           ..
/Users/danongohou/Desktop/P6/Images/26989e846c2096a5b59b59cbea2cc7ab.jpg    1
/Users/danongohou/Desktop/P6/Images/9a02f44389bda4c60e7dc23947dbca58.jpg    1
/Users/danongohou/Desktop/P6/Images/79b78739b0ae84780001fec304ce036c.jpg    1
/Users/danongohou/Desktop/P6/Images/91e22428c0dd8871288ba5dac35a7382.jpg    1
/Users/danongohou/Desktop/P6/Images/f2f027ad6a6df617c9f125173da71e44.jpg    1
Name: count, Length: 1050, dtype: int64

In [20]:
data.label_name.value_counts()

label_name
Home Furnishing                 150
Baby Care                       150
Watches                         150
Home Decor and Festive Needs    150
Kitchen and Dining              150
Beauty and Personal Care        150
Computers                       150
Name: count, dtype: int64

In [2]:
import os
import shutil
# Dossier train data
base_dir = '/Users/danongohou/Desktop/P6/train_val_test_datasets/'
os.mkdir(base_dir)

In [25]:
import os
import random
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

# Vérifier si les dossiers existent, sinon les créer
if not os.path.exists('train_set'):
    os.makedirs('train_set')
if not os.path.exists('test'):
    os.makedirs('test_set') 

# Charger les données
df = pd.read_csv('data_cls.csv')
df.image_path = df['image_path']
df.label_name = df['label_name']

# Diviser les données en ensembles d'entraînement et de test
train_data, test_data =  train_test_split(df, test_size=0.2, stratify=df.label_name, random_state=42)

# Sauvegarder les datasets dans les dossiers appropriés
train_data.to_csv('train_set/train_data.csv', index=False)
test_data.to_csv('test_set/test_data.csv', index=False)

print('train_data : ', train_data.shape[0])
print('test_data : ', test_data.shape[0])

In [26]:
# Sauvegarder les datasets dans les dossiers appropriés
train_data.to_csv('train_set/train_data.csv', index=False)
test_data.to_csv('test_set/test_data.csv', index=False)

In [27]:
print('train_data : ', train_data.shape[0])
print('test_data : ', test_data.shape[0])

train_data :  840
test_data :  210


In [28]:
train_data.head()


,image,image_path,label_name
799,5188dd28a627807407d93549580afb74.jpg,/Users/danongohou/Desktop/P6/Images/5188dd28a6...,Home Furnishing
324,a43fbae655e5d13215b9dd65339fa9d4.jpg,/Users/danongohou/Desktop/P6/Images/a43fbae655...,Computers
173,6b043b37c38f081d96886acb5acfbdf5.jpg,/Users/danongohou/Desktop/P6/Images/6b043b37c3...,Kitchen and Dining
391,7fd12776e6d62da6e1dd3cbc9882ee5f.jpg,/Users/danongohou/Desktop/P6/Images/7fd12776e6...,Computers
670,d06bb04b096c77dc3302eaf2db336f96.jpg,/Users/danongohou/Desktop/P6/Images/d06bb04b09...,Baby Care


## Étape 0 : Division du dataset en train et test

In [40]:
import os
import random
from PIL import Image
from sklearn.model_selection import train_test_split

# Chemin du dossier contenant les images
image_folder = '/Users/danongohou/Desktop/P6/Images/'

# Vérifier si les dossiers existent, sinon les créer
if not os.path.exists('train_set'):
    os.makedirs('train_set')
if not os.path.exists('test_set'):
    os.makedirs('test_set')

# Liste des noms de fichiers des images
image_files = os.listdir(image_folder)

# Diviser les données en données d'entraînement et de test
train_files, test_files = train_test_split(image_files, test_size=0.2)

# Sauvegarder les images d'entraînement
for filename in train_files:
    source_path = os.path.join(image_folder, filename)
    destination_path = os.path.join('train_set', filename)
    image = Image.open(source_path)
    image.save(destination_path)

# Sauvegarder les images de test
for filename in test_files:
    source_path = os.path.join(image_folder, filename)
    destination_path = os.path.join('test_set', filename)
    image = Image.open(source_path)
    image.save(destination_path)


## Verification de la division

In [119]:
train_folder = './train_set'  # Dossier d'entraînement contenant les images
test_folder = './test_set'    # Dossier de test contenant les images


# Compter le nombre d'images dans le dossier d'entraînement
train_image_names = os.listdir(train_folder)
num_train_images = len(train_image_names)

# Compter le nombre d'images dans le dossier de test
test_image_names = os.listdir(test_folder)
num_test_images = len(test_image_names)

print("Nombre d'images dans le dossier d'entraînement : ", num_train_images)
print("Nombre d'images dans le dossier de test : ", num_test_images)

Nombre d'images dans le dossier d'entraînement :  840
Nombre d'images dans le dossier de test :  210


## Étape 1 : Augmentation du train data

In [145]:

import os
import cv2
import pandas as pd

image_dir = './train_set/'
label_file = 'train_labels/train_lab.csv'

labels = pd.read_csv(label_file)
image_paths = []
image_labels = []

# Get a list of image files
image_files = [filename for filename in os.listdir(image_dir) if filename.endswith('.jpg')]

for filename in image_files:
    image_path = os.path.join(image_dir, filename)
    filename_without_extension = os.path.splitext(filename)[0]
    label_mask = labels['image_name'] == filename
    if label_mask.any():
        label = labels.loc[label_mask, 'label_name'].values[0]
        image_paths.append(image_path)
        image_labels.append(label)

# Verify the results
for image_path, label in zip(image_paths, image_labels):
    print(image_path, label)
    




In [138]:
import pandas as pd
from sklearn.utils import shuffle
import os
import cv2
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

image_paths = './train_set/'
labels = pd.read_csv('train_labels/train_lab.csv')

batch_size = 32  # Define the batch size

def data_flow_fct(image_paths, labels, datagen, data_type=None):
    
    data_flow = datagen.flow_from_dataframe(data, directory='',
                                             x_col='image_path', y_col='label_name',
                                             weight_col=None, target_size=(224, 224),
                                             classes=None, class_mode='categorical',
                                             batch_size=batch_size, shuffle=True, seed=42,
                                             subset=data_type)
    return data_flow

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25,
    preprocessing_function=preprocess_input)

train_flow = data_flow_fct(image_paths, labels, datagen, data_type='training')
val_flow = data_flow_fct(image_paths, labels, datagen, data_type='validation')
test_flow = data_flow_fct(test_data, labels, datagen, data_type=None)


ValueError: If using all scalar values, you must pass an index

In [121]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [59]:
print(len(image_paths))
print(len(labels))

840
7


## Étape 2 : Preprocess du train data augmenté

In [ ]:
import numpy as np
import os
import cv2
from PIL import Image, ImageOps

# Chemin vers le dossier d'entrée contenant les images
input_path = './train_set'

# Chemin vers le dossier de sortie
output_path = './train_set_Processed/'

# Créer le dossier de sortie s'il n'existe pas
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Parcourir toutes les images dans le dossier d'entrée
for filename in os.listdir(input_path):
    # Vérifier si le fichier est une image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        
        # Charger l'image avec OpenCV
        image = cv2.imread(os.path.join(input_path, filename))

        # Correction de l'exposition avec PILS
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        image = ImageOps.autocontrast(image)
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        # Conversion en niveau de gris de l'image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Equalization de l'histogramme de l'image
        image = cv2.equalizeHist(image)
        
        # Correction du contraste 
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(3,3))
        image = clahe.apply(image)
    
        # Réduction de bruit avec OpenCV
        image = cv2.fastNlMeansDenoising(image, h=10)
    
        # Ajout de BLUR avec OpenCV
        image = cv2.GaussianBlur(image, (5, 5), 0)
        
        # Réduction de dimension avec OpenCV
        image = cv2.resize(image, (224, 224))
        
        # Enregistrer l'image prétraitée dans le dossier de sortie
        output_filename = os.path.splitext(filename)[0] + '.jpg'
        output_file_path = os.path.join(output_path, output_filename)
        cv2.imwrite(output_file_path, image)


## Étape 3 : Validation Croisée sur le Train data

In [ ]:
import os
import cv2
from sklearn.model_selection import KFold

# Répertoire contenant les images d'entraînement
train_directory = './train_set/'

# Liste pour stocker les chemins d'accès aux images
image_paths = []

# Parcourir les fichiers dans le répertoire d'entraînement et ajouter les chemins d'accès aux images
for filename in os.listdir(train_directory):
    if filename.endswith(".jpg"):  # Vérifier l'extension de fichier appropriée
        image_path = os.path.join(train_directory, filename)
        image_paths.append(image_path)



### Validation croisée k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_val_indices = [(train_idx, val_idx) for train_idx, val_idx in kf.split(image_paths)]

val_accs = []
test_accs = []

for fold, (train_indices, val_indices) in enumerate(train_val_indices):
    print(f'Fold {fold+1}/{5}')

    # Obtenir les chemins d'accès aux images pour le fold
    train_image_paths = [image_paths[i] for i in train_indices]
    val_image_paths = [image_paths[i] for i in val_indices]


## Étape 4 : Entrainement du modèle sur Train Data augmenté

In [ ]:
# Création du modèle
with tf.device('/gpu:0'): 
    model = create_model_fct()

# Création du callback
model_save_path = f"./model_fold{fold+1}_best_weights.h5"
checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks_list = [checkpoint, es]

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
with tf.device('/gpu:0'): 
    history = model.fit(train_flow,
                        validation_data=val_flow,
                        batch_size=batch_size,
                        epochs=50,
                        callbacks=callbacks_list,
                        verbose=1)
        
# Enregistrer les données de l'entraînement pour tracer les graphes
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

## Étape 5 : Évaluation du modèle sur Train Dataset

In [ ]:
### évaluation du modèle
# Score du dernier epoch
loss, accuracy = model.evaluate(train_flow, verbose=True)
print("Training Accuracy   : {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(val_flow, verbose=True)
print("Validation Accuracy :  {:.4f}".format(accuracy))



## Étape 6 : Évaluation du modèle sur Train Dataset

In [ ]:
# Score de l'epoch optimal
model.load_weights(model_save_path)
loss, val_accuracy = model.evaluate(val_flow, verbose=False)
print("Validation Accuracy :  {:.4f}".format(val_accuracy))
loss, test_accuracy = model.evaluate(test_flow, verbose=False)
print("Test Accuracy       :  {:.4f}".format(test_accuracy))


In [ ]:
val_accs.append(val_accuracy)
test_accs.append(test_accuracy)

In [ ]:
print(f'Average validation accuracy over {k} folds: {np.mean(val_accs)}')
print(f'Average test accuracy over {k} folds: {np.mean(test_accs)}')

## Étape 7 : Graphique Loss et Accuracy

In [ ]:
# Tracer les graphes de Loss et d'Accuracy
import matplotlib.pyplot as plt

plt.plot(train_loss)
plt.plot(val_loss)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(train_acc)
plt.plot(val_acc)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()